## 1ª. Etapa - Seleção e Preparação dos Dados

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### 1. Loop para construir a tabela principal

In [2]:
# Selecionando os dados a serem trabalahdos neste projeto de análise de dados
# Criando um lista para utilizar o loop for para cointruir a tabela
lista = list(range(2001, 2021,1))
# Criando um dataframe vazio para receber as tabelas
tabelas = pd.DataFrame([])
# Executando um loop para obter os dados das colunas de todas as tabelas e concatenar em um novo arquivo
for item in lista:
    tabela = pd.read_csv(f'Files_1996_2020/SINASC_{item}.CSV', sep=';', encoding='ISO-8859-1')
    tabela_final = tabela[['DTNASC','IDADEMAE','GRAVIDEZ', 'PARTO', 'SEXO', 'PESO','IDANOMAL','CODANOMAL']]
    tabelas = pd.concat([tabelas, tabela_final])


### 1.1 Reformatação dos dados e filtro dos dados para análise

In [3]:
# Criando a coluna ANO com apply e lambda
tabelas['ANO'] = tabelas['DTNASC'].apply(lambda x: str(x)[-4:])
# removendo a coluna DTNASC já que criamos a coluna ano, para não aumentar muito o tamanho do Dataset
tabelas = tabelas.drop(columns='DTNASC')
# Visualizando os anos únicos
tabelas.ANO.unique()

array(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020'], dtype=object)

In [4]:
# Definimos a idade mínima em 10 e máxima em 50 para este estudo, considerando pesquisas sobre natalidade 
# que definem esse período como o mais comum para a gestação.
# Aplicando o filtro de idades
tabelas = tabelas.query('IDADEMAE>=10 & IDADEMAE<=50')

In [5]:
# IDADEMAE   = Criar uma coluna com a distribuição por faixa de idade entre 10 e 50 anos
tabelas['FAIXA_ETARIA'] = pd.cut(tabelas.IDADEMAE, bins=[0,20,30,40,50],
                                 labels=['0-20','21-30','31-40','41-50'])

In [6]:
# Conforme documentacao
# GRAVIDEZ = Tipo de gravidez: 1– Única; 2– Dupla; 3– Tripla ou mais; 9– Ignorado
tabelas.GRAVIDEZ.value_counts(dropna=False)

1.0    57514853
2.0     1129495
NaN       71182
3.0       36944
9.0       18812
Name: GRAVIDEZ, dtype: int64

In [7]:
# GRAVIDEZ = Tipo de gravidez: 1– Única; 2– Dupla; 3– Tripla ou mais; 9– Ignorado
tabelas.GRAVIDEZ = tabelas.GRAVIDEZ.replace(9.0,0)
tabelas.GRAVIDEZ.value_counts(dropna=False)

1.0    57514853
2.0     1129495
NaN       71182
3.0       36944
0.0       18812
Name: GRAVIDEZ, dtype: int64

In [8]:
tabelas.PARTO.value_counts(dropna=False)

2.0    29394948
1.0    29272956
NaN       73551
9.0       29831
Name: PARTO, dtype: int64

In [9]:
# PARTO = Tipo de parto: 1– Vaginal; 2– Cesário; 9– Ignorado
parto = {1:'Normal',
         2:'Cesária',
         9:'Não Informado'}
tabelas.PARTO = tabelas.PARTO.map(parto)
tabelas.PARTO.value_counts(dropna=False)

Cesária          29394948
Normal           29272956
NaN                 73551
Não Informado       29831
Name: PARTO, dtype: int64

In [10]:
tabelas.SEXO.value_counts(dropna=False)

1    28567909
2    27196232
M     1526728
F     1451492
0       27388
9         878
I         659
Name: SEXO, dtype: int64

In [11]:
# SEXO = Sexo: 1- M – Masculino; 2- F – Feminino; 0- I – ignorado
sexo = {1:'Masculino',
        'M':'Masculino',
        2:'Feminino',
        'F':'Feminino',
        0:'Não Informado',
        'I':'Não Informado',
        9:'Não Informado'}
tabelas.SEXO = tabelas.SEXO.map(sexo)
tabelas.SEXO.value_counts(dropna=False)

Masculino        30094637
Feminino         28647724
Não Informado       28925
Name: SEXO, dtype: int64

In [12]:
tabelas.IDANOMAL.value_counts(dropna=False)

2.0    55622819
NaN     1601506
9.0     1119718
1.0      426609
8.0         633
5.0           1
Name: IDANOMAL, dtype: int64

In [13]:
# IDANOMAL - Anomalia identificada: 1– Sim; 2– Não; 9– Ignorado 
idanomal = {1.0:'Sim',
            2.0:'Não',
            5.0: np.nan,
            8.0: np.nan,
            9.0: 'Não Informado'
           }
tabelas.IDANOMAL = tabelas.IDANOMAL.map(idanomal)
tabelas.IDANOMAL.value_counts(dropna=False)

Não              55622819
NaN               1602140
Não Informado     1119718
Sim                426609
Name: IDANOMAL, dtype: int64

In [14]:
# Salvando o arquivo gerado em uma tabela CSV
tabelas.to_csv('tabelas_v1.csv', index=False)